In [1]:
# extract files
#file = open("./tiny-imagenet-200/wnids.txt", "r+")
#print (file)

In [2]:
import cv2
from os import listdir
import matplotlib.pyplot as plt
from matplotlib import image
import numpy as np
from scipy import ndimage, misc

In [3]:
import tensorflow as tf
#import keras
#from keras.models import Sequential
#from keras.layers import Dense, ZeroPadding2D, Conv2D, BatchNormalization, Activation, AveragePooling2D, DepthwiseConv2D, Flatten, Reshape, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization, Activation
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [13]:
##
# load all images in Training Directory
##
loaded_images = list()
loaded_y = list()
X_test = list()
img_size =128
!pwd
path ='./stanford dogs/'
for name in listdir(path):
    print(name)
clas_folder = path +'/Images/'

dict={}
for name in listdir(clas_folder):
    print(name)
list_clas_folder = listdir(clas_folder)
#list_clas_folder.remove('.DS_Store')

/home/ecbm4040/stanford results


FileNotFoundError: [Errno 2] No such file or directory: './stanford dogs/'

In [ ]:
num_classes = len(list_clas_folder)

for i in range(num_classes):
    dict[str(i)] = list_clas_folder[i]
    folder_name = list_clas_folder[i]
    image_path = clas_folder + folder_name
    
    for filename in listdir(image_path):
        img_data = cv2.resize(cv2.imread(image_path + filename), (img_size, img_size),interpolation = cv2.INTER_AREA)
        loaded_images.append(img_data)
        loaded_y.append(i)
print("Data Loaded")
plt.imshow(loaded_images[7])

In [ ]:
seq = np.arange(len(loaded_y))
np.random.shuffle(seq)
print("shuffling..")
len(seq)
X_train1 = list()
y_train1 = list()
#loaded_images = loaded_images*(1.0/255)
for i in range(len(seq)):
    X_train1.append(loaded_images[seq[i]])
    y_train1.append(loaded_y[seq[i]])

In [ ]:
X_train = np.asarray(X_train1)
plt.imshow(X_train[3])


In [ ]:
X_train  = X_train*(1./255)
y_train = np.asarray(y_train1)
print("shuffling done...")
print("All data Loaded..")
plt.imshow(X_train[3])

In [ ]:
from tensorflow.keras.utils import to_categorical as one_hot
y_train = one_hot(y_train)
print("y_train_shape = " , y_train.shape)
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_test = X_train[-20000:-10000]
y_test = y_train[-20000:-10000]
X_train = X_train[:-20000]
y_train = y_train[:-20000]
print("X_val_shape = " ,X_val.shape)
print("X_train_shape = " ,X_train.shape)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
plt.imshow(X_train[7])
datagen.fit(X_train)
plt.imshow(X_train[7])


In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Conv2D,MaxPooling2D,Flatten,Dropout,BatchNormalization, Activation
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
#from keras.models import Sequential
#from keras.layers import Dense, ZeroPadding2D, Conv2D, BatchNormalization, Activation, AveragePooling2D, DepthwiseConv2D, Flatten, Reshape, GlobalAveragePooling2D
#import keras
print(tf.__version__)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ZeroPadding2D, Conv2D, BatchNormalization, Activation, AveragePooling2D, DepthwiseConv2D, Flatten, Reshape, GlobalAveragePooling2D
#import keras
from tensorflow.keras.applications.mobilenet import MobileNet

base_model = MobileNet(include_top = False, weights ='imagenet', input_shape = (128,128,3))
x = base_model.output
x = Conv2D(64, kernel_size=1, strides=1, activation = 'relu')(x)
x = Flatten()(x)
x = Dense(256, activation ='relu')(x)
x = BatchNormalization()(x)
x = Dense(128, activation ='relu')(x)
x = BatchNormalization()(x)
pred = Dense(200, activation ='softmax')(x)

In [ ]:
model = Model(inputs = base_model.input, outputs = pred)
print(len(model.layers))
for layer in model.layers[:-8]:
    layer.trainable =False
for layer in model.layers[-8:]:
    layer.trainable = True

In [ ]:
model.summary()

In [ ]:
#model = Model(inputs=model.input, outputs =preds)
from tensorflow.keras.optimizers import Adam as Adam

adam = Adam(lr=0.001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#from tensorflow.keras.utils import multi_gpu_model
#from keras.application import Xception

#par_model = multi_gpu_model(model, gpus = 8)
#par_model.compile(optimizer=adam,
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])

In [ ]:
#history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val))
model.fit_generator(datagen.flow(X_train, y_train, batch_size=256),
                    steps_per_epoch=len(X_train) / 256, epochs=30, validation_data = (X_val, y_val))

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=128)

In [ ]:
# Save the model
model.save('trained_model_mobilenet_64_all_data.h5')

In [ ]:
# Recreate the exact same model purely from the file
from keras.models import load_model
#new_model = load_model('path_to_my_model.h5')
classifierLoad = tf.keras.models.load_model('path_to_my_model.h5')

In [ ]:
classifierLoad.summary()